In [48]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
import datetime 
from bs4 import BeautifulSoup
import re

### Read, clean and convert loanbook

In [77]:
df = pd.read_csv('loanbook.csv')

# only 2012 dates onwards
df = df[pd.to_datetime(df['loan_accepted_date']) > datetime.date(year=2012, month=1, day=1)]

# ignore repaying (possible over simplification)
df = df[df['status'] != 'loan: repaying']

# select only status text
df['status'] = df['status'].str[6:]

# independent successful overall repayment
df['success'] = df['status'] == 'repaid'

# clean purpose and sector
df['loan_purpose'] = df['loan_purpose'].str.lower().str.replace('/', ' ')
df['sector'] = df['sector'].str.lower().str.replace('/', ' ')

# risk band simplification
df['credit_band'] = df['credit_band'].str[:2]

print(df['status'].value_counts())
print(df['success'].value_counts())
df.tail(5)

repaid       7005
defaulted    1131
late          343
Name: status, dtype: int64
True     7005
False    1474
Name: success, dtype: int64


,id,status,credit_band,loan_purpose,sector,business_type_name,region_name,loan_amount,recoveries,interest_rate,...,num_loan_parts,loan_accepted_date,date_repaid,whole_loan,related_auctions,repayment_type,loan_guaranteed,year_incorporated,security_taken,success
26541,30464,repaid,A,working capital,arts & entertainment,Limited Company,London,10500.0,0.0,9.0,...,1,2016-12-30,2021-12-30,WL,NaN,Amortising,1.0,2011.0,No asset security,True
26798,30725,repaid,A,expansion growth,finance,Limited Company,Scotland,52000.0,0.0,8.5,...,1,2017-01-10,2020-01-10,WL,"30913, 13150",Amortising,1.0,2010.0,No asset security,True
26874,30802,repaid,B,expansion growth,manufacturing and engineering,Limited Company,North East,15675.0,0.0,9.5,...,1,2017-01-12,2019-01-12,WL,"9106, 22694, 16978, 13346, 31578, 6712",Amortising,1.0,1993.0,No asset security,True
28437,32365,repaid,A+,property development,property and construction,Limited Company,London,490000.0,0.0,10.0,...,6194,2017-02-09,2018-02-09,PL,"32937, 32876, 32634, 32593, 32482, 32409, 1544...",Interest only,1.0,2015.0,First charge,True
28504,32433,repaid,A+,working capital,consumer services,Partnership (less than 4 partners),North West,26020.0,0.0,7.5,...,1,2017-02-13,2022-02-13,WL,NaN,Amortising,NaN,NaN,No asset security,True


### Generate one hot encodings for text terms

This is to understand the `loan_purpose` and `sector` fields, which are written inconsistently.

First extract the most common terms, then generate a dataframe with one hot encoding per row.

In [79]:
# extract terms across dataframe column
def term_extraction(df_col, min_size=1, min_count=5):
    words = df_col.tolist()
    words = [x if type(x) is str else '' for x in words]
    words_join = ' '.join(words).split(' ')
    terms = Counter(words_join).most_common(20)
    
    terms = [t for t, c in terms if (len(t) > min_size and c > min_count)]
    
    return terms, words

purpose_terms, purpose_words = term_extraction(df['loan_purpose'])
sector_terms, sector_words = term_extraction(df['sector'])
    
    
# generate one hot dataframe representation
def term_onehot_generator(words, index, terms=purpose_terms, prepend='purpose_'):
    m = np.zeros((len(words), len(terms)))

    for i, row in enumerate(words):
        m[i,:] = [int(t in row) for t in terms]

    return pd.DataFrame(data=m, index=index, columns=[prepend + t for t in terms])

df_purpose = term_onehot_generator(purpose_words, df.index, terms=purpose_terms, prepend='purpose_')
df_sector = term_onehot_generator(sector_words, df.index, terms=sector_terms, prepend='sector_')

### Generate X and y dataframes for estimation

In [80]:
# one hot credit dataframe
df_credit_onehot = pd.get_dummies(df['credit_band'])

# select continuous columns
df_cont = df[['loan_amount', 'interest_rate', 'term']]

# concatenate dataframes for X
X = pd.concat([df_purpose, df_sector, df_credit_onehot, df_cont], axis=1)

# concatenate encoding for success and status
y = pd.concat([pd.get_dummies(df['status']), df['success']], axis=1)

print(df_purpose.shape, df_credit_onehot.shape, df_cont.shape)
print(X.shape, y.shape)

(8479, 18) (8479, 6) (8479, 3)
(8479, 46) (8479, 4)


### Learn prediction model

We use a basic ExtraTrees Regression model from scikit-learn

In [83]:
clf = ExtraTreesRegressor()

clf.fit(X,y)

print("Feature importances")
clf_fi = clf.feature_importances_
clf_fi_order = np.argsort(clf.feature_importances_)[::-1]
for c, v in zip(np.array(X.columns.values)[clf_fi_order], clf_fi[clf_fi_order]):
    print('%s: %.4f' % (c, v))

Feature importances
loan_amount: 0.3316
interest_rate: 0.2933
term: 0.0671
sector_retail: 0.0173
C : 0.0166
purpose_capital: 0.0158
B : 0.0157
sector_other: 0.0144
purpose_expansion: 0.0130
A : 0.0126
sector_and: 0.0124
purpose_working: 0.0123
purpose_growth: 0.0122
sector_wholesale: 0.0114
sector_automotive: 0.0102
sector_healthcare: 0.0099
sector_construction: 0.0086
sector_property: 0.0085
D: 0.0085
sector_hospitality: 0.0083
A+: 0.0081
sector_engineering: 0.0078
sector_leisure: 0.0075
sector_i.t: 0.0067
sector_manufacturing: 0.0064
sector_telecommunications: 0.0064
sector_support: 0.0063
sector_logistics: 0.0055
sector_professional: 0.0051
sector_business: 0.0051
sector_transport: 0.0049
purpose_other: 0.0048
purpose_property: 0.0045
purpose_asset: 0.0040
purpose_purchase: 0.0032
E: 0.0030
purpose_refinancing: 0.0028
purpose_loan: 0.0021
purpose_finance: 0.0020
purpose_short: 0.0011
purpose_term: 0.0010
purpose_residential: 0.0007
purpose_payment: 0.0007
purpose_tax: 0.0004
purpose

### Predictor for new data

Generates a dataframe for some new data and prints the outcomes

In [87]:
def predict(
    purpose = 'Working Capital Loan', 
    sector = 'Manufacturing and Engineering', 
    location = 'London',
    risk = 'A+', 
    amount = 52000, 
    term = 60, 
    rate = 7.5):
    print('\n%s - %s | %s | %d | %dm | %.1f%%' % (purpose, sector, risk, amount, term, rate))
    ln_purpose = term_onehot_generator([purpose], [0], terms=purpose_terms, prepend='purpose_')
    ln_sector = term_onehot_generator([sector], [0], terms=sector_terms, prepend='sector_')
    ln_credit_onehot = pd.DataFrame([[int(c == risk.ljust(2)) for c in df_credit_onehot.columns.values]], columns=df_credit_onehot.columns.values)
    
    ln_cont = pd.DataFrame([[amount, rate, term]], columns=df_cont.columns.values)

    ln = pd.concat([ln_purpose, ln_sector, ln_credit_onehot, ln_cont], axis=1)
    
    proba = clf.predict(ln)[0]
    
    print('Return %.1f%% > %.1f%% (%.1f%%)' % (rate *.9, rate *.9 * (1.0-proba[0]), rate))
    print(' | '.join(['%.0f%% %s' % (p*100.0, o.upper()) for o, p in zip(y.columns.values, proba)]))

### Pre-defined examples to show some outcomes

In [88]:
predict(
    purpose = 'Working Capital Loan', 
    sector = 'Manufacturing and Engineering', 
    location = 'London',
    risk = 'A+', 
    amount = 52000, 
    term = 60, 
    rate = 7.5)

predict(
    purpose = 'Expansion And Growth Loan ', 
    sector = 'Retail', 
    location = 'London',
    risk = 'B', 
    amount = 262500, 
    term = 60, 
    rate = 10.5)

predict(
    purpose = 'Expansion And Growth Loan ', 
    sector = 'Wholesale',
    location = 'South East',
    risk = 'B', 
    amount = 21200, 
    term = 60, 
    rate = 10.5)

predict(
    purpose = 'Working Capital Loan ', 
    sector = 'Retail', 
    location = 'South East',
    risk = 'B', 
    amount = 21200, 
    term = 60, 
    rate = 10.5)


Working Capital Loan - Manufacturing and Engineering | A+ | 52000 | 60m | 7.5%
Return 6.8% > 6.8% (7.5%)
0% DEFAULTED | 0% LATE | 100% REPAID | 100% SUCCESS

Expansion And Growth Loan  - Retail | B | 262500 | 60m | 10.5%
Return 9.5% > 7.6% (10.5%)
20% DEFAULTED | 20% LATE | 60% REPAID | 60% SUCCESS

Expansion And Growth Loan  - Wholesale | B | 21200 | 60m | 10.5%
Return 9.5% > 9.5% (10.5%)
0% DEFAULTED | 10% LATE | 90% REPAID | 90% SUCCESS

Working Capital Loan  - Retail | B | 21200 | 60m | 10.5%
Return 9.5% > 9.5% (10.5%)
0% DEFAULTED | 10% LATE | 90% REPAID | 90% SUCCESS


# Predict from an HTML file

To use this function, save your current Funding Circle loanbook page `https://www.fundingcircle.com/lend/loan-requests/` to `fc-lb.html`.

Each loan row will be predicted for. 

In [75]:
soup = BeautifulSoup(open('fc-lr.html'), 'html.parser')
for elem in soup.find_all(id=re.compile('auction')):
#     print(elem)
    purpose = elem('a')[1].contents[0].strip()
    sector_location_uid = elem('span')[0].contents[0].strip().split(',')
    sector = sector_location_uid[0].strip()
    location = sector_location_uid[1].split('-')[0].strip()
    risk = elem('td')[2].contents[0].strip()
    amount = int(elem('td')[3].contents[0].strip().replace(',','')[1:])
    term = int(elem('td')[4].contents[0].strip())
    rate = float(elem('td')[5].contents[0].strip()[:-1])
    
    uid = sector_location_uid[1].split('-')[1].strip()
    
    predict(purpose, sector, risk, amount, term, rate)
    print(uid)


Expansion And Growth Loan - Retail | B | 262500 | 60m | 10.5%
Return 9.5% > 7.6% (10.5%)
20% DEFAULTED | 0% LATE | 80% REPAID | 80% STATUS_SUCCESS
33006

Expansion And Growth Loan - Wholesale | B | 146865 | 60m | 10.5%
Return 9.5% > 8.5% (10.5%)
10% DEFAULTED | 0% LATE | 90% REPAID | 90% STATUS_SUCCESS
33115

Expansion And Growth Loan - Wholesale | B | 212000 | 60m | 10.5%
Return 9.5% > 7.6% (10.5%)
20% DEFAULTED | 0% LATE | 80% REPAID | 80% STATUS_SUCCESS
33098

Property Development In Uxbridge 7 - Property and Construction | A | 93900 | 12m | 10.0%
Return 9.0% > 9.0% (10.0%)
0% DEFAULTED | 0% LATE | 100% REPAID | 100% STATUS_SUCCESS
33110

Expansion And Growth Loan - Wholesale | B | 41800 | 24m | 9.5%
Return 8.6% > 8.6% (9.5%)
0% DEFAULTED | 0% LATE | 100% REPAID | 100% STATUS_SUCCESS
33111

Working Capital Loan - Leisure & Hospitality | A | 15750 | 60m | 9.0%
Return 8.1% > 6.5% (9.0%)
20% DEFAULTED | 0% LATE | 80% REPAID | 80% STATUS_SUCCESS
33114

Expansion and Growth Loan - Retai